In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

name_schema = StructType([
  StructField("forename", StringType(), False),
  StructField("surname", StringType(), False),
])
drivers_schema = StructType([
  StructField("driverId", IntegerType(), False),
  StructField("driverRef", StringType(), False),
  StructField("number", IntegerType(), True),
  StructField("code",StringType(), False),
  StructField("name", name_schema, False),
  StructField("dob", DateType(), True),
  StructField("nationality", StringType(), True),
  StructField("url", StringType(), True),
])

In [0]:
drivers_df = spark.read.json("dbfs:/mnt/f1dlnagsa/raw/drivers.json", schema=drivers_schema)

In [0]:
from pyspark.sql.functions import current_timestamp, concat, lit
drivers_final_df = drivers_df.drop("url")\
        .withColumnRenamed("driverId", "driver_id")\
        .withColumnRenamed("driverRef", "driver_ref")\
        .withColumn("ingestion_date",current_timestamp())\
                .withColumn("name",concat("name.forename", lit(" "), "name.surname"))

In [0]:
drivers_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.drivers")

In [0]:
dbutils.notebook.exit("Success")